<a href="https://colab.research.google.com/github/Hani1-2/DeepLearningAssignmnt/blob/master/Rumour_Tweets_Classification_Using_BERT_%26_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv("/content/Clean_Datasetmod.csv",encoding='ISO-8859-1')
df.head(5)

,id,text,source,created_at,user_id,username,user_description,account_date,user_location,followers,followings,retweet_count,fav_count,lang,event,target
0,544297950134816768,picture gunman inside lindt cafe martin place ...,tweetdeck,Mon Dec 15 01:08:38 +0000 2014,9609632,The Australian,news australian newspaper australian online,Mon Oct 22 23:57:33 +0000 2007,sydney australia,230648,395,226,51,en,sydneysiege-all-rnr-threads,1
1,544301900682629121,police take suspect martinplace unclear whethe...,tweetdeck,Mon Dec 15 01:24:20 +0000 2014,35466620,Herald Sun,part conversation news melbourne herald sun ne...,Sun Apr 26 13:44:23 +0000 2009,melbourne australia,116475,35970,338,133,en,sydneysiege-all-rnr-threads,1
2,544504183341064192,escaped sydneysiege hostage elly chen top stud...,twitter web client,Mon Dec 15 14:48:08 +0000 2014,22594051,The Star,news updates malaysia top english language dai...,Tue Mar 03 06:19:00 +0000 2009,kuala lumpur malaysia,442770,81,182,130,en,sydneysiege-all-rnr-threads,1
3,544377502563459072,ultra capitalism backflip uber rethinks fee hi...,tweetdeck,Mon Dec 15 06:24:45 +0000 2014,8197942,SBS News,sbs world news bringing global national news m...,Wed Aug 15 06:40:45 +0000 2007,australia,122339,885,119,108,en,sydneysiege-all-rnr-threads,1
4,544274934835707905,several people held hostage sydney cafe austra...,socilflow,Sun Dec 14 23:37:11 +0000 2014,742143,BBC News (World),world news features analysis uk news follow bb...,Thu Feb 01 07:44:29 +0000 2007,london uk,8175508,61,762,113,en,sydneysiege-all-rnr-threads,1


## BERT:

BERT (Bidirectional Encoder Representations from Transformers) is a recent paper published by researchers at Google AI Language. It has caused a stir in the Machine Learning community by presenting state-of-the-art results in a wide variety of NLP tasks.

BERT makes use of Transformer, an attention mechanism that learns contextual relations between words (or sub-words) in a text. In its vanilla form, Transformer includes two separate mechanisms — an encoder that reads the text input and a decoder that produces a prediction for the task

[link text](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270)

##Data PreProcessing



*   Frist we separate our input and target variables
*   Then we found the values_counts which our data is highly unbiased



In [3]:
df_x = df[['text']]  # input features
df_target = df[['target']] # target features

 There are 4022 entries for label ‘0’ and only 1436 entries for label ‘1’. This is not a fair deal. Hence, in the next section, we will work on balancing this dataset by using the undersampling method. 

In [4]:
df_target.value_counts() # no of tweets rumour or non-rumour

target
0         4022
1         1436
dtype: int64

## Balancing the Dataset

There are many techniques to balance a dataset like- SMOTE, cluster abundant classes, resampling, and many others. Of these, the simplest is undersampling. In **undersampling**, the entries of the majority class are dropped till it becomes equal to the number of entries in the minority class.

[techniques to handle imbalance data](https://www.youtube.com/watch?v=JnlM4yLFNuo)

In [6]:
df_0_class = df_target == 0
df_1_class = df_target == 1 
df_0_class_undersampled = df_0_class.sample(df_1_class.shape[0])
df = pd.concat([df_0_class_undersampled, df_1_class], axis=0)

In [7]:
df.value_counts() 
# since here the target is to see the power of BERT, so we are not going in balancing details

target
False     5458
True      5458
dtype: int64

## Splitting the Dataset

 We are going to divide the dataset into two parts namely- the training dataset and the test dataset. There is another classification possible known as validation dataset. For the sake of simplicity, I will split the dataset in two using sklearn package.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x,df_target, stratify=df_target)

In [9]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

## BERT Preprocessor & Encoder

Here we download BERT processor and encoder via URL

Recommended link: If you wanna know about more of BERT encoders. We use BERT Basic with 12 enoders 

[Bert processor](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3)

[Bert Encoders](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4)


In [11]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# BERT Layers and Neural Networks

we have BERT layers (the first three) followed by the neural network layer. We have included a dropout layer and eventually, the output layer will classify whether the given text is rumour or not.

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text-layer')
preprocessed_text = preprocess(text_input)
outputs = encoder(preprocessed_text)
d_layer = tf.keras.layers.Dropout(0.1, name="dropout-layer")(outputs['pooled_output'])
d_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(d_layer)
model = tf.keras.Model(inputs=[text_input], outputs = [d_layer])

In [13]:
m= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=m)

Since this is tensorflow model, we have to provide here activation function which is sigmoid

## Model Evaluation

Model training will take alot of time. Since Google Colab with runtime soecified as GPU will be preferred to run the neural network.
The model is ready to learn


In [14]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
128/128 [==============================] - 51s 315ms/step - loss: 0.5763 - accuracy: 0.7305 - precision: 0.2174 - recall: 0.0093
Epoch 2/10
128/128 [==============================] - 43s 336ms/step - loss: 0.5371 - accuracy: 0.7415 - precision: 0.6267 - recall: 0.0436
Epoch 3/10
128/128 [==============================] - 43s 338ms/step - loss: 0.5145 - accuracy: 0.7430 - precision: 0.5984 - recall: 0.0706
Epoch 4/10
128/128 [==============================] - 43s 336ms/step - loss: 0.4968 - accuracy: 0.7518 - precision: 0.6266 - recall: 0.1402
Epoch 5/10
128/128 [==============================] - 43s 337ms/step - loss: 0.4890 - accuracy: 0.7505 - precision: 0.5875 - recall: 0.1746
Epoch 6/10
128/128 [==============================] - 43s 337ms/step - loss: 0.4804 - accuracy: 0.7611 - precision: 0.6364 - recall: 0.2145
Epoch 7/10
128/128 [==============================] - 43s 338ms/step - loss: 0.4754 - accuracy: 0.7552 - precision: 0.5904 - recall: 0.2275
Epoch 8/10
128/128 [

In [15]:
model.evaluate(X_test, y_test) # Accuracy is about 76% on testing unseen data

43/43 [==============================] - 17s 329ms/step - loss: 0.4537 - accuracy: 0.7626 - precision: 0.5907 - recall: 0.3175


[0.45373570919036865,
 0.7626373767852783,
 0.590673565864563,
 0.3175487518310547]

## Classification Report

Only better way to understand our model is confusion matrix and classification report.

In [16]:
import numpy as np
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_predicted)
matrix

43/43 [==============================] - 16s 344ms/step


array([[927,  79],
       [245, 114]])

In [17]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.79      0.92      0.85      1006
           1       0.59      0.32      0.41       359

    accuracy                           0.76      1365
   macro avg       0.69      0.62      0.63      1365
weighted avg       0.74      0.76      0.74      1365

